In [ ]:
import warnings
warnings.filterwarnings('ignore')
from tensorflow import keras
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import os
import random
import zipfile
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from keras.applications.densenet import DenseNet121
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.callbacks import Callback
from shutil import copyfile, rmtree

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")
!unzip -q "/content/gdrive/My Drive/Dataset.zip"
!ls

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
replace Dataset/Normal/Normal-1.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace Dataset/Normal/Normal-10.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
chest_xray.h5  gdrive	    tb-xray-database
Dataset        sample_data  vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5


In [ ]:
normal_path = "../content/Dataset/Normal"
tb_path = "../content/Dataset/Tuberculosis"
print(f"Normal X-rays {len(os.listdir(normal_path))}")
print(f"TB X-rays {len(os.listdir(tb_path))}")

Normal X-rays 3500
TB X-rays 700


In [ ]:
# make directories for storing training and test sets
try:
    parent_dir = './tb-xray-database/'
    os.mkdir(parent_dir)
    training_path = os.path.join(parent_dir, "training")
    testing_path = os.path.join(parent_dir, "testing")
    os.mkdir(training_path)
    os.mkdir(testing_path)
    os.mkdir(os.path.join(training_path, "normal"))
    os.mkdir(os.path.join(training_path, "tb"))
    os.mkdir(os.path.join(testing_path, "normal"))
    os.mkdir(os.path.join(testing_path, "tb"))
except OSError:
    print("couldn't make it")
    pass

In [ ]:
# split into training and test sets

def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):
    path_list = os.listdir(SOURCE)# file names in SOURCE
    length = len(path_list)
    count = 0
    for file in random.sample(path_list, length):
        file_path = os.path.join(SOURCE, file)
        if (os.path.getsize(file_path) != 0):
            if (count < SPLIT_SIZE*length):
                copyfile(file_path, os.path.join(TRAINING, file))
                count+=1
            else:
                copyfile(file_path, os.path.join(TESTING, file))


NORMAL_SOURCE_DIR = "../content/Dataset/Normal"
TRAINING_NORMAL_DIR = os.path.join(training_path, "normal")
TESTING_NORMAL_DIR = os.path.join(testing_path, "normal")
TB_SOURCE_DIR = "../content/Dataset/Tuberculosis"
TRAINING_TB_DIR = os.path.join(training_path, "tb")
TESTING_TB_DIR = os.path.join(testing_path, "tb")

split_size = .7
split_data(NORMAL_SOURCE_DIR, TRAINING_NORMAL_DIR, TESTING_NORMAL_DIR, split_size)
split_data(TB_SOURCE_DIR, TRAINING_TB_DIR, TESTING_TB_DIR, split_size)

In [ ]:
print(len(os.listdir(TRAINING_NORMAL_DIR)))
print(len(os.listdir(TESTING_NORMAL_DIR)))
print(len(os.listdir(TRAINING_TB_DIR)))
print(len(os.listdir(TESTING_TB_DIR)))

2450
1050
490
210


In [ ]:
IMAGE_SIZE = [512, 512]

train_path = './tb-xray-database/training'
valid_path = './tb-xray-database/testing'

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")
!unzip -q "/content/gdrive/My Drive/vgg16.zip"
!ls

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Dataset  sample_data	   vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5
gdrive	 tb-xray-database


In [ ]:
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights ='../content/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5', include_top=False)

In [ ]:
for layer in vgg.layers:
    layer.trainable = False

In [ ]:
folders = glob('./tb-xray-database/training/*')
x = Flatten()(vgg.output)

In [ ]:
prediction = Dense(len(folders), activation='softmax')(x)
# create a model object
model = Model(inputs=vgg.input, outputs=prediction)
# view the structure of the model
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 512, 512, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 512, 512, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 512, 512, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 256, 256, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 256, 256, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 256, 256, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 128, 128, 128)     0     

In [ ]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)




# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('./tb-xray-database/training',
                                                 target_size = (512, 512),
                                                 batch_size = 10,
                                                 class_mode = 'categorical')




test_set = test_datagen.flow_from_directory('./tb-xray-database/testing',
                                            target_size = (512, 512),
                                            batch_size = 10,
                                            class_mode = 'categorical')

Found 2940 images belonging to 2 classes.
Found 1260 images belonging to 2 classes.


In [ ]:
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=1,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

294/294 [==============================] - 11294s 38s/step - loss: 0.5182 - accuracy: 0.9299 - val_loss: 0.1365 - val_accuracy: 0.9810


In [ ]:
import tensorflow as tf
from keras.models import load_model

model.save('chest_xray.h5')

In [ ]:
from keras.models import load_model
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
model=load_model('chest_xray.h5')

In [ ]:

img=image.load_img('./tb-xray-database/testing/Tuberculosis-1.png',target_size=(512,512))

FileNotFoundError: ignored

In [ ]:
x=image.img_to_array(img)
x=np.expand_dims(x, axis=0)
img_data=preprocess_input(x)
classes=model.predict(img_data)
result=int(classes[0][0])
if result==0:
    print("Person is Affected ByTuberculosis")
else:
    print("Result is Normal")

NameError: ignored